In [1]:
import os
import sys

sys.path.insert(0, "\\".join(os.path.abspath(os.curdir).split("\\")[:-1:]))
from pathlib import Path

from copy import deepcopy

from src.utils.utils import *
from src.utils.imageProcessing import *

# Pré processar os metadados

### CBIS-DDSM

In [2]:
json_names = ['calc_case_description_test_set', 'calc_case_description_train_set', 
              'mass_case_description_test_set', 'mass_case_description_train_set']

path_metadata = "../metadata/mamografia/extracted_metadata/csv_and_dicom_DSSM"

# Cria o objeto do banco de dados
database = {"name": "CBIS-DDSM",
            "exameType": "Mamografia",
            "description": "Banco de dados gerado pelo processamento do banco DDSM",
            "imageQuality": set(),
            "imageType": "DDSM",
            "sourceLink": "https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=22516629",
            }

studies = []
list_metadata = []
images = []

# Pega e junta todos os metadados
for json_name in json_names:
    list_metadata = list_metadata + load_json(json_name, path_metadata)
    
for i, metadata in enumerate(list_metadata):
    metadata_csv = rename_keys(metadata["metadata_csv"])
    metadata_dicom = metadata["metadata_dicom_files"]["original"][0]
    
    database["imageQuality"].add(int(metadata_dicom["bits_stored (0028, 0101)"]))
    resizedImagePath = f"D:\\mathe\\Documents\BancoDados_IC\\resized_images\\CBIS-DDSM\\image_resized_CBIS-DDSM_{i}.png"
    
    directory = Path(metadata_csv["original_image_path"])
    images_paths = list(directory.rglob(f"*.dcm*"))
    images_paths = list(map(str, images_paths))
    metadata_csv["original_image_path"] = images_paths[0]
    
    """ image_array = extract_image_dicom(metadata_csv["original_image_path"])
    resize_image(image_array, (255, 255), save=True, path=resizedImagePath, image_type="png") """
    
    image = {
             "patient_id": metadata_csv["patient_id"],
             "leftOrRightBreast": metadata_csv["left_or_right_breast"],
             "orientation": metadata_csv["image_view"],
             "width": metadata_dicom["columns (0028, 0011)"],
             "height": metadata_dicom["rows (0028, 0010)"],
             "imagePath": metadata_csv["original_image_path"],
             "resizedImagePath": resizedImagePath,
             "imageQuality": metadata_dicom["bits_stored (0028, 0101)"],
             "classification": {
                                 "pathology": metadata_csv["pathology"],
                                 "biRads": metadata_csv["bi_rads"],
                                 "breastDensity": metadata_csv["breast_density"],
                                 "abnormalityType": metadata_csv["abnormality_type"],
                                 "subtlety": metadata_csv["subtlety"]
                                },
            }

    images.append(image)

if len(database["imageQuality"]) == 1:
       database["imageQuality"] = database["imageQuality"][0]
else:
    database["imageQuality"] = list(database["imageQuality"]) 
    
database["images"] = images

save_json("metadata", database, "./mongodb_database") 

### CMMD

In [3]:
"""#CMMD e INBREAST     
database_names = ['CMMD', 'INBREAST']
json_names = ['csv_and_dicom_CMMD', 'csv_and_dicom_INBREAST']

for database_name, json_name in zip(database_names, json_names):
    list_metadata = load_json(json_name, "../metadata/mamografia/extracted_metadata")
    
    for metadata in list_metadata: 
        new_metadata = {}
        new_metadata['database_name'] = database_name
        new_metadata['study_name'] = metadata['study_name']
        
        metadata_csv = rename_keys(metadata['metadata_csv'])
    
        new_metadata = new_metadata | metadata_csv
        new_metadata['metadata_dicom_files'] = metadata['metadata_dicom_files']
        
        studies.append(new_metadata)
    
save_json("metadata", studies, "./mongodb_database") """

'#CMMD e INBREAST     \ndatabase_names = [\'CMMD\', \'INBREAST\']\njson_names = [\'csv_and_dicom_CMMD\', \'csv_and_dicom_INBREAST\']\n\nfor database_name, json_name in zip(database_names, json_names):\n    list_metadata = load_json(json_name, "../metadata/mamografia/extracted_metadata")\n    \n    for metadata in list_metadata: \n        new_metadata = {}\n        new_metadata[\'database_name\'] = database_name\n        new_metadata[\'study_name\'] = metadata[\'study_name\']\n        \n        metadata_csv = rename_keys(metadata[\'metadata_csv\'])\n    \n        new_metadata = new_metadata | metadata_csv\n        new_metadata[\'metadata_dicom_files\'] = metadata[\'metadata_dicom_files\']\n        \n        studies.append(new_metadata)\n    \nsave_json("metadata", studies, "./mongodb_database") '